# Домашнее задание 1

### Описание

В вашем распоряжении датасет с русскоязычными отзывами о мобильных телефонах с выставленным рейтингом от 1 до 5.
Ключевая задача – обучить любую модель регрессии (или классификации, если решите таким путём пойти) из пакетов scikit, XGBoost, LightGBM, CatBoost.


Необходимая метрика:

1. Со звёздочкой (дополнительный балл) – MAE <= 0.5
2. Минимальное допустимое значение – МАЕ <= 1.0

### Что необходимо сделать

1. Откройте датасет
2. Разделите на обучение и тест
3. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 
4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).
5. Вывести ТОП-50 наиболее частотных токенов:
- только для униграмм
- только для биграмм
- только для триграмм
- для всех вариантов n-грамм одновременно

Напишите, какие наблюдения и выводы есть.
6. Повторите пункт 5, только отдельно для отзывов с рейтингом «4-5», «3» и «1-2». Есть ли ключевые отличия? Есть кандидаты на попадание в список стоп-слов?

7. Составьте список своих ключевых слов, для помощи можно использовать nltk и punctuations из string. 
8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer (экспериментируйте с параметрами min_df  и max_df). 
9. Обучение одну или несколько моделей машинного обучения на разных представлениях данных
10. Валидируйте модель. Если модель соответствует условиям метрик, то работа завершена. В ином случае, экспериментируйте, начиная с пункта 7. 
11. По всем попыткам обучить качественную модель пишите свои выводы и замечания, почему так получилось.


## Комментарии исполнителя

Я изменил порядок 2 и 3 пункта, потому что так намного удобнее

## 0. Импорт библиотк, определение констант

In [21]:
import os
import pandas as pd
import numpy as np
import spacy

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pymorphy3 import MorphAnalyzer
from nltk import ngrams

In [22]:
tqdm.pandas()


In [23]:
#!python -m spacy download ru_core_news_sm

## 1. Откройте датасет

In [24]:
if os.path.exists("data_lemma.csv"):
    df = pd.read_csv("data_lemma.csv", engine='python')
else:    
    df = pd.read_csv("data.csv")

In [25]:
df.shape

(459497, 5)

In [26]:
df.head(10)

,Unnamed: 0,index,Review,Rating,lemma
0,0,0.0,3D Touch просто восхитительная вещь! Заряд дер...,5.0,3d touch просто восхитительный вещь! заряд дер...
1,1,1.0,"Отключается при температуре близкой к нулю, не...",4.0,"отключаться при температура близкий к нулю, не..."
2,2,2.0,"В Apple окончательно решили не заморачиваться,...",3.0,"в apple окончательно решить не заморачиваться,..."
3,3,3.0,Постарался наиболее ёмко и коротко описать все...,4.0,постараться наиболее ёмко и коротко описать вс...
4,4,4.0,Достойный телефон. Пользоваться одно удовольст...,5.0,достойный телефон. пользоваться один удовольст...
5,5,5.0,6s gold 64gb,5.0,6s gold 64gb
6,6,6.0,Мой первый айфон. Скажу честно- Эппл ранее ник...,5.0,мой первый айфон. сказать честно- эппл ранее н...
7,7,7.0,Мне очень понравилась эта модель! Во-первых кл...,5.0,я очень понравиться этот модель! во-первых кла...
8,8,8.0,Долгое время пользовалась iPhone 5s 16Gb. Он м...,5.0,долгий время пользоваться iphone 5s 16gb. он я...
9,9,9.0,"Раньше был Samsung Galaxy Alpha SM-G850F 32Gb,...",4.0,ранний быть samsung galaxy alpha sm-g850f 32gb...


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459497 entries, 0 to 459496
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  459497 non-null  object 
 1   index       458849 non-null  float64
 2   Review      458849 non-null  object 
 3   Rating      456817 non-null  float64
 4   lemma       456817 non-null  object 
dtypes: float64(2), object(3)
memory usage: 17.5+ MB


In [28]:
df = df.dropna().reset_index()

In [29]:
df['Rating'].value_counts()

Rating
5.0    227560
4.0    106314
3.0     52949
2.0     35608
1.0     34386
Name: count, dtype: int64

In [30]:
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

In [31]:
df['Rating'].value_counts(normalize=True)

Rating
5.0    0.498143
4.0    0.232728
3.0    0.115909
2.0    0.077948
1.0    0.075273
Name: proportion, dtype: float64

## 2. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 

### spacy

In [32]:
spacy_nlp = spacy.load("ru_core_news_sm")

In [33]:
def spacy_lemmatize_text(text):
    try:
        doc = spacy_nlp(text)
    except:
        print(f'{text}')
        return ""
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

### pymorphy3

In [34]:
pymorphy3_analyzer = MorphAnalyzer()

In [35]:
def pymorphy3_lemmatize_text(text):
    lemmas = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    return (' '.join(lemmas))

### Лемматинизируем

In [36]:
if not "lemma" in df.columns:
    print('Лемматизируем текст');
    df["lemma"] = df["Review"].progress_apply(lambda row: pymorphy3_lemmatize_text(row))
    df.to_csv("data_lemma.csv", index=False)

## 2. Разделите на обучение и тест

In [37]:
train, test = train_test_split(df, test_size=0.2, stratify=df['Rating'])
print(train.shape)
print(test.shape)

(365453, 6)
(91364, 6)


## 4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).

### Униграммы

In [38]:
list(ngrams(train['lemma'], 1)) # униграммы

[('если бы быть такой возможность, взять бы другой телефон',),
 ('проблема с русификация решиться настройка "формат региона" ура, теперь недостаток нет!!!быть тонкости.. пока не понял, можно ли устанавливать приложение на флешку. с флешка видный только музыка и видео. документ офис на флешка не видит.',),
 ('телефон отличный, только качество отвратное, после айфон 6 , как то разочаровать именно в качество материал и сборки. сетка сверху протираться , кнопка домой жуткий пластиковый с ужасный неприятный нажатием, дисплей корявый приклеен, зазор между корпус и дисплей не равномерные, звук динамика ужасен, в общий только камера и дисплей, ну ещё дизайн , всё остальной так себе.',),
 ('если для вы качество камера не являться главным, то одназначный на данный момент - это хороший сенсор',),
 ('на недостаток не заморачиваюсь: телефон падать в лужу, на асфальт много раз. и на последок я он утопил, в вода быть минута 15. просушить фен и подержать денёк на батарее. включил-всё работает, никакой